In [9]:
import sqlite3
import os
from sqlite3 import Error
import pandas as pd

In [2]:
import requests
from bs4 import BeautifulSoup

C:\Users\beneo\AppData\Roaming\Python\Python38\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.6) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
# Здесь указываем рабочую директорию.
os.chdir('A:/MIPT')

In [10]:
# Загружаем полную таблицу с датасетом
df = pd.read_csv("final_dataset.csv")

C:\Users\Ekaterina\AppData\Local\Temp\ipykernel_14252\1482730268.py:2: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17,18,20,21,25,26,27,33,34,35,36,37,38,39,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("final_dataset.csv")


In [3]:
df.head()

,name,name_norm,Website,country_code,city,state_code,status,founded_at,closed_at,first_investment_at,...,TOP_avg_multiplicator,TOP_success_strategy_diversity_index,TOP_follow_on_index,TOP_readiness_take_younger_startups,team_size,employeeRange,followerCount,positionCount,stage,best_company
0,Wetpaint,wetpaint,http://wetpaint-inc.com,USA,Seattle,WA,operating,2005-10-17,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Flektor,flektor,http://www.flektor.com,USA,Culver City,CA,acquired,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,There,there,http://www.there.com,USA,San Mateo,CA,acquired,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,MYWEBBO,mywebbo,http://www.mywebbo.com,NaN,NaN,NaN,operating,2008-07-26,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,THE Movie Streamer,the movie streamer,http://themoviestreamer.com,NaN,NaN,NaN,operating,2008-07-26,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
# Берем только те строки, где у компании есть web сайт.
dataset = df[df['Website'].notna()]

In [11]:
comp_dict = {}
for name, site in zip(dataset['name'], dataset['Website']):
    comp_dict[name] = site

In [13]:
len(comp_dict)

480737

In [5]:
# Делаем список имен компаний, в том порядке, в котором они даны в таблице.
company_list = []
for i in dataset['name']:
    company_list.append(i)
len(company_list)

508262

In [6]:
len(set(company_list))

480737

In [7]:
# Делаем соответствующий список web-адресов компаний.
website_list = []
for i in dataset['Website']:
    website_list.append(i)
len(website_list)

508262

In [8]:
len(set(website_list))

499987

In [36]:
# Функция, которая осуществляет парсинг. Функция берет сайт википедии и прибавляет к нему имя компании. 
# Если такого сайта нет, при парсинге добавляются наиболее типичные окончания из списка additions (ниже).
# Функция работает только для имен без пробелов.

def parsing(u):
    url = 'https://en.wikipedia.org/wiki/'+u
    print(url)
    response = requests.get(url)
    print(response.status_code)
    soup = BeautifulSoup(response.text, "lxml")
    txt = soup.find('tbody').text
    begin = ['Traded as','Founded','Revenue ','Operating income ','Net income ','Total assets ','Total equity ']
    end = [':',';','(','(','(','(','(',]
    features = u+'%'+url+'%'
    for i in range(len(begin)):
        try:
            x = txt.find(begin[i])
            txt = txt[x+len(begin[i]):]
            b = txt.find((end[i]))
            word = txt[:b].strip()
            features += word
            features +='%'
        except Exception:
            features += 'None'
            features +='%'
    if len(features) < 300:
        return features
    else:
        features = u+'%'+url+'%'
        return features

In [37]:
additions=[',_Inc.','_(company)','_Platforms']

In [40]:
# !!! Здесь указываем, какой дипазон компаний берем для парсинга
name_parsing_list = company_list[:100000]
web_parsing_list = website_list[:100000]

In [ ]:
# Здесь проводится сам парсинг.
with open('full_wiki_parsing.csv','a', encoding = 'utf-8') as f:
    for k in range(len(company_list)):
        try:
            features = parsing(str(name_parsing_list[k]))
            f.write(str(web_parsing_list[k])+'%')
            f.write(features)
            f.write("\n")
        except Exception:
            for l in additions:
                try:
                    features = parsing(name_parsing_list[k]+l)
                    f.write(str(web_parsing_list[k])+'%')
                    f.write(features)
                    f.write("\n")
                except Exception:
                    print(f'{str(name_parsing_list[k])+l} was not included')